<a href="https://colab.research.google.com/github/AnishVelala/AI-ML-ASL-Sign-Language-Learning-Tool/blob/main/Copy_of_ASL_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import cv2

DATA_DIR ='./data'

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)


letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
dataset_size = 1000

cap = cv2.VideoCapture(0)

for letter in letters:
    if not os.path.exists(os.path.join(DATA_DIR, letter)):
        os.makedirs(os.path.join(DATA_DIR, letter))

    print('Collecting data for class {}'.format(letter))

    done = False
    while True:
        ret, frame = cap.read()
        cv2.putText(frame, 'Ready Press "Q" ! :)', (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3,
                    cv2.LINE_AA)
        cv2.imshow('frame', frame)
        if cv2.waitKey(25) == ord('q'):
            break

    counter = 0
    while counter < dataset_size:
        ret, frame = cap.read()
        cv2.imshow('frame', frame)
        cv2.waitKey(25)
        cv2.imwrite(os.path.join(DATA_DIR, letter, '{}.jpg'.format(counter + 2000)), frame)
        print(counter)
        counter += 1

cap.release()
cv2.destroyAllWindows()

'''
1. get a series of images in a directory
2. create a directory
3. for each letter in alphabet, create a directory and then run the data collection (
- 500 for left
- 500 for right
)

'''




In [ ]:
import os
import pickle
import requests
import numpy as np
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt

mp_hands = mp.solutions.hands

# Clone the repository

count = 0

# Define the base URL of the GitHub repository
base_url = 'https://raw.githubusercontent.com/SameerHaq349/ASL-Dataset/main/Images/'

# Function to read image from URL
def read_image_from_url(url):
    resp = requests.get(url)
    img_arr = np.asarray(bytearray(resp.content), dtype=np.uint8)
    img = cv2.imdecode(img_arr, -1)
    return img

# Initialize Mediapipe Hands model
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

data = []
labels = []

# Iterate through directories in the cloned repository
for dir_ in os.listdir('data'):
    if os.path.isdir(os.path.join('data', dir_)):
        for img_file in os.listdir(os.path.join('data', dir_)):
            img_path = os.path.join('data', dir_, img_file)
            img = cv2.imread(img_path)

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            results = hands.process(img_rgb)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    data_aux = []
                    for landmark in hand_landmarks.landmark:
                        x = landmark.x
                        y = landmark.y
                        z = landmark.z

                        data_aux.append(x)
                        data_aux.append(y)
                        data_aux.append(z)





                    data.append(data_aux)
                    labels.append(dir_)
                    count = count + 1

                    if(count == 999):
                        count = 0


# Save the data and labels using pickle
with open('data.pickle', 'wb') as f:
    pickle.dump({'data': data, 'labels': labels}, f)

print({'data': data, 'labels': labels})

In [ ]:
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

import pandas as pd



data_dict = pickle.load(open('./data.pickle', 'rb'))


data = np.asarray(data_dict['data'])
labels = np.asarray(data_dict['labels'])

#print(type(data[1][0]))
#print(type(labels[0]))

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)


model = RandomForestClassifier()

model.fit(pd.DataFrame(x_train), y_train)

y_predict = model.predict(x_test)
score = accuracy_score(y_predict, y_test)

print(y_predict)
print(score)
print(x_test)
print(x_train)
print(y_train)
print(y_test)

f = open('model.p1', 'wb')
pickle.dump({'model': model}, f)
f.close()

#print(data)
#print(labels)

#for each dir, go ahead and flip image and then add label twice

Cloning into 'ASL-Dataset'...
remote: Enumerating objects: 23825, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 23825 (delta 0), reused 3 (delta 0), pack-reused 23822
Receiving objects: 100% (23825/23825), 1.43 GiB | 20.44 MiB/s, done.
Updating files: 100% (24002/24002), done.


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:




import pickle
import cv2
import mediapipe as mp
import numpy as np

model_dict = pickle.load(open('./model.p1', 'rb'))
model = model_dict['model']

cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)


labels_dict = {
 'A': 'A', 'B': 'B', 'L': 'L', 'C': 'C', 'D': 'D', 'E': 'E',
 'F': 'F', 'G': 'G', 'H': 'H', 'I': 'I', 'J': 'J', 'K': 'K',
 'M': 'M', 'N': 'N', 'O': 'O', 'P': 'P', 'Q': 'Q', 'R': 'R',
 'S': 'S', 'T': 'T', 'U': 'U', 'V': 'V', 'W': 'W', 'X': 'X',
 'Y': 'Y', 'Z': 'Z'
}
while True:




    ret, frame = cap.read()

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

        for hand_landmarks in results.multi_hand_landmarks:
            data_aux = []
            x_ = []
            y_ = []
            z_ = []
            for i in range(len(hand_landmarks.landmark)):
                X = hand_landmarks.landmark[i].x
                Y = hand_landmarks.landmark[i].y
                Z = hand_landmarks.landmark[i].z
                data_aux.append(X)
                data_aux.append(Y)
                data_aux.append(Z)
                x_.append(X)
                y_.append(Y)
                z_.append(Z)

            x1 = int(min(x_) * W) - 10
            y1 = int(min(y_) * H) - 10

            x2 = int(max(x_) * W) - 10
            y2 = int(max(y_) * H) - 10

            prediction = model.predict([np.asarray(data_aux)])

            predicted_character = labels_dict[prediction[0]]


            print(predicted_character)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,0,0), 4)
            cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0,0,0), 3, cv2.LINE_AA)

    cv2.imshow('frame', frame)
    cv2.waitKey(1)

cap.release()
cv2.destroyAllWindows()

